In [1]:
!pip install gdown
!gdown --id 1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn

/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn
From (redirected): https://drive.google.com/uc?id=1-1EjRHAq-4nhXIJ9-OobWmMXavl1B4zn&confirm=t&uuid=39eaa9aa-2697-416f-b184-53f60ef51d82
To: /kaggle/working/data.pkl
100%|███████████████████████████████████████| 1.24G/1.24G [00:05<00:00, 221MB/s]


In [2]:
import pickle
from tqdm import tqdm
import tensorflow as tf
import numpy as np

from preprocess import data_preparation
from preprocess import signal_cutter
from preprocess import data_filtering
from preprocess import resampling
from preprocess import RP_3D
from preprocess import resize_volume
from preprocess import RP_final_data
from preprocess import label_creator
from preprocess import including_data
from metrics import result

2024-07-07 08:01:53.409021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 08:01:53.409136: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 08:01:53.541621: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Read dictionary pkl file
with open('/kaggle/working/data.pkl', 'rb') as fp:
    dataset = pickle.load(fp)

In [4]:
ds = data_preparation(dataset)

# shortening
ds = signal_cutter(ds, d_length=1000)

# denoising
ds = data_filtering(ds)

# resampling
ds = resampling(ds, desired_points = 100)

100%|██████████| 10344/10344 [00:01<00:00, 9299.15it/s]


In [5]:
arr = {'NSR':'426783006', 'AF':'164889003', 'IAVB':'270492004', 'LBBB':'164909002', 'RBBB':'59118001', 'SB':'426177001', 'STach':'427084000'}
labels = label_creator(data_preparation(dataset), arr)
x_train, y_train, x_test, y_test = including_data(labels, ds)

100%|██████████| 10344/10344 [00:00<00:00, 941676.92it/s]
10344it [00:07, 1329.09it/s]
10344it [00:04, 2290.11it/s]


In [6]:
data_2D = []
for i in tqdm(x_train):
    tmp_2D = RP_3D(i)
    data_2D.append(np.reshape(tmp_2D, (99,99,12)))
x_train_2d = np.array(data_2D)

data_2D = []
for i in tqdm(x_test):
    tmp_2D = RP_3D(i)
    data_2D.append(np.reshape(tmp_2D, (99,99,12)))
x_test_2d = np.array(data_2D)

100%|██████████| 1126/1126 [00:05<00:00, 195.78it/s]


In [7]:
print(np.shape(x_train_2d))
print(np.shape(x_test_2d))

(5000, 99, 99, 12)
(1126, 99, 99, 12)


In [8]:
# Model

def Radhakrishnana_model():
    i = tf.keras.layers.Input(shape=(99,99,12))

    x = tf.keras.layers.Conv2D(3, kernel_size=(30,45))(i)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Conv2D(5, kernel_size=(20,25))(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Conv2D(7, kernel_size=(12,12))(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Conv2D(9, kernel_size=(8,6))(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Reshape((15, 33*9))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(500)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Dense(200)(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    x = tf.keras.layers.Dense(7, activation = 'sigmoid')(x)

    return tf.keras.models.Model(i, x)

m = Radhakrishnana_model()
m.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 99, 99, 12)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 70, 55, 3)      │        48,603 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 70, 55, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 70, 55, 3)      │            12 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 70, 55, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 51, 31, 5)      │         7,505 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 51, 31, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 51, 31, 5)      │            20 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 51, 31, 5)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 40, 20, 7)      │         5,047 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 40, 20, 7)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 40, 20, 7)      │            28 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 40, 20, 7)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 33, 15, 9)      │         3,033 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 33, 15, 9)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 33, 15, 9)      │            36 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 33, 15, 9)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 15, 297)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 15, 200)        │       318,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 500)            │     1,500,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_4 (ReLU)                  │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 500)            │             

 Total params: 1,984,791 (7.57 MB)

 Trainable params: 1,984,743 (7.57 MB)

 Non-trainable params: 48 (192.00 B)

In [9]:
# Training

## Compiling
def CompModel(model):
    model.compile(optimizer=tf.keras.optimizers.Adam() , 
                loss=tf.keras.losses.BinaryCrossentropy() , metrics=['acc'])
    lr_sch = tf.keras.callbacks.LearningRateScheduler(
        lambda epochs: 1e-3 * 10 ** (-epochs/100.0))
    return model, lr_sch

model = Radhakrishnana_model()
model, lr_sch = CompModel(model)

history = model.fit(x_train_2d, y_train.values,
            epochs=100,
            callbacks = [lr_sch],
            verbose = 1)

Epoch 1/100


2024-07-07 08:04:12.256012: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_3_1/dropout_6_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


157/157 ━━━━━━━━━━━━━━━━━━━━ 17s 45ms/step - acc: 0.3676 - loss: 0.4084 - learning_rate: 0.0010
Epoch 2/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - acc: 0.6327 - loss: 0.2632 - learning_rate: 9.7724e-04
Epoch 3/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.6374 - loss: 0.2518 - learning_rate: 9.5499e-04
Epoch 4/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.6792 - loss: 0.2226 - learning_rate: 9.3325e-04
Epoch 5/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.6800 - loss: 0.2156 - learning_rate: 9.1201e-04
Epoch 6/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.6985 - loss: 0.2055 - learning_rate: 8.9125e-04
Epoch 7/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - acc: 0.7083 - loss: 0.1994 - learning_rate: 8.7096e-04
Epoch 8/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.7358 - loss: 0.1838 - learning_rate: 8.5114e-04
Epoch 9/100
157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - acc: 0.7227 - loss: 0.1862 - learning_rate: 8.3176e-04
Epoch 10/100
157

In [10]:
# Evaluating

pred = model.predict(x_test_2d)
prediction = np.zeros(np.shape(pred))
prediction[pred > 0.5] = 1
result(y_test, prediction)

36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
SNR
accuracy: 85.7 precision: 74.35 sensitivity: 81.84 specificity: 87.42 f1_score: 77.92
_______
AF
accuracy: 90.5 precision: 47.54 sensitivity: 27.88 specificity: 96.87 f1_score: 35.15
_______
IAVB
accuracy: 84.19 precision: 26.6 sensitivity: 18.66 specificity: 93.04 f1_score: 21.93
_______
LBBB
accuracy: 96.71 precision: 74.07 sensitivity: 40.0 specificity: 99.35 f1_score: 51.95
_______
RBBB
accuracy: 92.27 precision: 75.47 sensitivity: 35.09 specificity: 98.72 f1_score: 47.91
_______
SB
accuracy: 93.07 precision: 85.31 sensitivity: 87.14 specificity: 95.04 f1_score: 86.22
_______
STach
accuracy: 94.94 precision: 82.38 sensitivity: 95.13 specificity: 94.89 f1_score: 88.3
_______
accuracy: 63.94 %
AUC: 75.08 %
f1-weighted: 68.35 %
f1-micro: 70.86 %
f1-macro: 58.48 %
